In [1]:
import os
import pathlib

import numpy as np
import tensorflow as tf

In [20]:
DATASET_PATH = 'barkmeow'

data_dir = pathlib.Path(DATASET_PATH)
print(data_dir)
commands = np.array(tf.io.gfile.listdir(str(data_dir)))

In [3]:
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(contents=audio_binary)
  return tf.squeeze(audio, axis=-1)
def get_label(file_path):
  parts = tf.strings.split(
      input=file_path,
      sep=os.path.sep)
  return parts[-2]

def get_waveform_and_label(file_path):
  label = get_label(file_path)
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform, label

def get_spectrogram(waveform):
  input_len = 16000
  waveform = waveform[:input_len]
  zero_padding = tf.zeros(
      [16000] - tf.shape(waveform),
      dtype=tf.float32)
  waveform = tf.cast(waveform, dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

def get_spectrogram_and_label_id(audio, label):
  spectrogram = get_spectrogram(audio)
  label_id = tf.argmax(label == commands)
  return spectrogram, label_id

def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)
  output_ds = files_ds.map(
      map_func=get_waveform_and_label,
      num_parallel_calls=tf.data.AUTOTUNE)
  output_ds = output_ds.map(
      map_func=get_spectrogram_and_label_id,
      num_parallel_calls=tf.data.AUTOTUNE)
  return output_ds

In [4]:
from tensorflow.keras.models import load_model

model = load_model("./model", compile=False)

In [22]:
sample_file = data_dir/'../test/oiseau1.wav'
print(sample_file)

sample_ds = preprocess_dataset([str(sample_file)])
print(sample_ds)

test\oiseau1.wav
<ParallelMapDataset shapes: ((None, 129, 1), ()), types: (tf.float32, tf.int64)>
